In [1]:
from syft import DataSubjectList

/home/e/anaconda3/envs/Hagrid/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import numpy as np

In [3]:
from syft import DataSubjectList

a = np.ones((2, 1, 5))
a

b = np.ones((10, 1, 1))
b

len(b)

(2, 1, 5) and (3, 1, 1) -> (2, 1, 5) -> (max(shape[0], shape[1]))


if len(a) < len(b) -> we have to create a bigger array
if len(a) == len(b) -> it fits!!!!
if len(a) > len(b) -> append some NaNs at the end of b
(3, 1, 5) and (2, 1, 1) -> (3, 1, 5)
ds1
ds2
NaN



(3, 50, 50) and 

dsl = DataSubjectList.from_objs(np.random.choice([10, 11, 12], (5, 5)))

dsl.one_hot_lookup, dsl.data_subjects_indexed.shape

dsl1 - > (5, 5)
dsl2 -> (3, 5)
index -> 0

dsl1.data_subjects_indexed.shape -> (1, 5)
dsl2 -> (3, 5)

output -> (3, 5, 5)

INPUT:
array([[1, 0, 1, 2, 0],
       [1, 0, 0, 0, 1],
       [0, 0, 2, 0, 1],
       [2, 0, 2, 1, 0],
       [2, 2, 1, 2, 0]])
      
OUTPUT:
array([[6, 6, 6, 6, 6],
       [1, 0, 0, 0, 1],
       [0, 0, 2, 0, 1],
       [2, 0, 2, 1, 0],
       [2, 2, 1, 2, 0]])

array([[7, 7, 7, 7, 7],
       [NAN, NAN, NAN, NAN, NAN],
       [NAN, NAN, NAN, NAN, NAN],
       [NAN, NAN, NAN, NAN, NAN],
       [NAN, NAN, NAN, NAN, NAN]])

array([[8, 8, 8, 8, 8],
       [NAN, NAN, NAN, NAN, NAN],
       [NAN, NAN, NAN, NAN, NAN],
       [NAN, NAN, NAN, NAN, NAN],
       [NAN, NAN, NAN, NAN, NAN]])



INPUT

child = [1, 2, 3]
dsl.dsi.shape != child.shape -> GammaTensor
(max(unique_data_subjects_per_data_point), 1, 3)

child = [1 2, 3
         4, 5, 6]
dsl.dsi.shape == child.shape -> PhiTensor

(0, 0, 0;
1, 1, 1)

(6, 6, 6,
7, 7, 7)

index = 0

In [4]:
def get_output_shape(shape1, shape2):
    if len(shape1) > len(shape2) or len(shape2) - len(shape1) > 1:
        raise NotImplementedError
        
    first_dim = max(shape1[0], shape2[0]) if len(shape1) == len(shape2) else shape2[0]
    other_dims = shape1 if len(shape1) < len(shape2) else shape1[1:]
    output_size = (first_dim, *other_dims)
    return output_size


def insert(dsl1: DataSubjectList, dsl2: DataSubjectList, index) -> DataSubjectList:
    
    output_shape = get_output_shape(dsl1.shape, dsl2.shape)
    
    # TODO: Broadcasting with [1] over multiple indices?
    dsl1_len = dsl1.shape[0] if len(dsl1.shape) == len(dsl2.shape) else 1
    dsl2_len = dsl2.shape[0]
    
    if dsl1.shape == dsl2.shape:
        return dsl2.copy()
    elif dsl1_len < dsl2_len:
        # We need to create a bigger array
        bigger_array = np.full(output_shape, np.nan)
        
        # Fill current dsl1 values into this bigger_array
        # TODO: Ask Madhava if there's a smarter/vectorized way to do this
        if dsl1_len == 1:
            bigger_array[0] = dsl1.data_subjects_indexed
        else:
            for i in range(dsl1_len):
                bigger_array[i] = dsl1[i]

        output_dsl = DataSubjectList(one_hot_lookup=dsl1.one_hot_lookup, data_subjects_indexed=bigger_array)
        # bigger_array[:, index] = dsl2.data_subjects_indexed.squeeze()
        dsl2_copy = dsl2.copy()
        dsl2_copy.data_subjects_indexed = dsl2_copy.data_subjects_indexed.squeeze()
        output_dsl[:, index] = dsl2_copy
        return output_dsl
        
    elif dsl1_len == dsl2_len:
        # it fits perfectly
        output_dsl = dsl1.copy()
        output_dsl[index] = dsl2
        return output_dsl
    else:
        # add some nans
        extra_nans = np.array([np.nan] * (dsl1_len - dsl2_len) * np.prod(dsl2.shape[1:])).reshape((dsl1_len - dsl2_len, *dsl2.shape[1:]))
        array_to_append = np.concatenate((dsl2.data_subjects_indexed, extra_nans)).squeeze()
        new_dsl = dsl2.copy()
        new_dsl.data_subjects_indexed = array_to_append
        
        output_dsl = dsl1.copy()
        output_dsl[:, index] = new_dsl
        return output_dsl
        

In [5]:
def test(dsl1_shape, dsl2_shape, index) -> DataSubjectList:
    dsl1 = DataSubjectList.from_objs(np.random.choice([80, 90, 100], size=dsl1_shape))
    dsl2 = DataSubjectList.from_objs(np.random.choice([50, 60, 70], size=dsl2_shape))
    # print(dsl1.one_hot_lookup, dsl1.data_subjects_indexed.shape)
    # print(dsl2.one_hot_lookup, dsl2.data_subjects_indexed.shape)
    
    output_dsl = insert(dsl1=dsl1, dsl2=dsl2, index=index)
    # print(output_dsl.data_subjects_indexed)
    return output_dsl

In [6]:
res = test((5, 5), (3, 1, 5), index=0)

In [7]:
res = test((5,), (5,), index=0)

In [8]:
res.one_hot_lookup

array([60, 70])

In [9]:
res = test((5, 5), (3, 1, 5), index=2)

In [10]:
res = test((5, 5), (3, 1, 5), index=-1)

In [11]:
res = test((3, 5, 5), (2, 1, 5), index=0)

In [12]:
res = test((5, 5), (3, 2, 5), index=[0, 1])

In [13]:
res.one_hot_lookup

array([ 80,  90, 100,  50,  60,  70])

In [14]:
res.data_subjects_indexed

array([[[ 4.,  4.,  3.,  5.,  3.],
        [ 4.,  3.,  3.,  3.,  3.],
        [ 0.,  1.,  1.,  0.,  1.],
        [ 1.,  0.,  2.,  0.,  1.],
        [ 2.,  0.,  0.,  0.,  2.]],

       [[ 3.,  4.,  3.,  5.,  3.],
        [ 3.,  3.,  3.,  4.,  3.],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan]],

       [[ 3.,  5.,  5.,  3.,  3.],
        [ 3.,  3.,  4.,  3.,  4.],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan]]])

In [15]:
from syft import GammaTensor

In [16]:
from syft import lazyrepeatarray

In [17]:
gt = GammaTensor(
    child=np.ones((5,5)), data_subjects=DataSubjectList.from_objs(np.random.choice(np.arange(7), (2, 5,5))), 
    min_vals=lazyrepeatarray(0, (5,5)), 
    max_vals=lazyrepeatarray(1, (5,5)))

In [18]:
gt.data_subjects.data_subjects_indexed

array([[[3, 4, 1, 6, 4],
        [0, 5, 2, 4, 1],
        [5, 2, 1, 5, 0],
        [1, 1, 0, 1, 6],
        [6, 4, 0, 4, 2]],

       [[1, 0, 2, 0, 4],
        [2, 3, 6, 4, 0],
        [5, 0, 6, 2, 4],
        [4, 2, 3, 2, 1],
        [6, 4, 4, 4, 5]]])

In [19]:
gt2 = GammaTensor(
    child=np.ones((5,5)) * 7, data_subjects=DataSubjectList.from_objs(np.random.choice(np.arange(10,17), (3,5,5))), 
    min_vals=lazyrepeatarray(0, (5,5)), 
    max_vals=lazyrepeatarray(1, (5,5))
)

In [20]:
gt2.data_subjects.data_subjects_indexed

array([[[6, 1, 1, 1, 3],
        [6, 6, 3, 4, 0],
        [4, 4, 2, 1, 4],
        [1, 4, 0, 1, 0],
        [4, 4, 6, 0, 3]],

       [[2, 0, 5, 1, 4],
        [5, 0, 3, 5, 2],
        [4, 2, 4, 5, 0],
        [5, 5, 5, 0, 3],
        [0, 4, 1, 5, 5]],

       [[4, 5, 4, 0, 6],
        [3, 6, 2, 4, 6],
        [2, 4, 4, 2, 6],
        [6, 4, 5, 0, 0],
        [2, 4, 2, 0, 6]]])

In [21]:
gt[0] = gt2[-1]

(2, 5, 5) (3, 5)
first dim 3
shapes:  (2, 5, 5) (3, 5)
len:  2 3
We are about to add nans
[[[nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]]

 [[nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]]

 [[nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]]]
else
(5, 5) (5, 5)
(3, 5, 5) (2, 5, 5)
before assignment [[[nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]]

 [[nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]]

 [[nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]]]
[[[ 3.  4.  1.  6.  4.]
  [ 0.  5.  2.  4.  1.]
  [ 5.  2.  1.  5.  0.]
  [ 1.  1.  0.  1.  6.]
  [ 6.  4.  0.  4.  2.]]


In [22]:
gt.data_subjects.data_subjects_indexed

array([[[11., 11., 13.,  7., 10.],
        [ 0.,  5.,  2.,  4.,  1.],
        [ 5.,  2.,  1.,  5.,  0.],
        [ 1.,  1.,  0.,  1.,  6.],
        [ 6.,  4.,  0.,  4.,  2.]],

       [[ 7., 11.,  8., 12., 12.],
        [ 2.,  3.,  6.,  4.,  0.],
        [ 5.,  0.,  6.,  2.,  4.],
        [ 4.,  2.,  3.,  2.,  1.],
        [ 6.,  4.,  4.,  4.,  5.]],

       [[ 9., 11.,  9.,  7., 13.],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan]]])

In [23]:
gt[0]

GammaTensor(child=array([7., 7., 7., 7., 7.]), data_subjects=<syft.core.adp.data_subject_list.DataSubjectList object at 0x7fc8e5fdadf0>, min_vals=<lazyrepeatarray data: 0 -> shape: (5,)>, max_vals=<lazyrepeatarray data: 7.0 -> shape: (5,)>, is_linear=True, func=<function no_op at 0x7fc8ec00b790>, id='1308222813', state={})

In [24]:
gt.data_subjects.one_hot_lookup

array([ 0,  1,  2,  3,  4,  5,  6, 10, 11, 12, 13, 14, 15, 16])

In [25]:
gt.data_subjects.data_subjects_indexed

array([[[11., 11., 13.,  7., 10.],
        [ 0.,  5.,  2.,  4.,  1.],
        [ 5.,  2.,  1.,  5.,  0.],
        [ 1.,  1.,  0.,  1.,  6.],
        [ 6.,  4.,  0.,  4.,  2.]],

       [[ 7., 11.,  8., 12., 12.],
        [ 2.,  3.,  6.,  4.,  0.],
        [ 5.,  0.,  6.,  2.,  4.],
        [ 4.,  2.,  3.,  2.,  1.],
        [ 6.,  4.,  4.,  4.,  5.]],

       [[ 9., 11.,  9.,  7., 13.],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan]]])

In [26]:
gt[0].data_subjects.data_subjects_indexed

array([[11., 11., 13.,  7., 10.],
       [ 7., 11.,  8., 12., 12.],
       [ 9., 11.,  9.,  7., 13.]])

In [27]:
gt2[0] = gt[2]

(3, 5, 5) (3, 5)
first dim 3
shapes:  (3, 5, 5) (3, 5)
len:  3 3
(3, 5)


In [28]:
gt.child

array([[7., 7., 7., 7., 7.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

In [29]:
gt2.child

array([[1., 1., 1., 1., 1.],
       [7., 7., 7., 7., 7.],
       [7., 7., 7., 7., 7.],
       [7., 7., 7., 7., 7.],
       [7., 7., 7., 7., 7.]])

In [30]:
gt2.data_subjects.one_hot_lookup

array([10., 11., 12., 13., 14., 15., 16.,  5.,  2.,  1.,  5.,  0.,  5.,
        0.])

In [31]:
gt.data_subjects.data_subjects_indexed

array([[[11., 11., 13.,  7., 10.],
        [ 0.,  5.,  2.,  4.,  1.],
        [ 5.,  2.,  1.,  5.,  0.],
        [ 1.,  1.,  0.,  1.,  6.],
        [ 6.,  4.,  0.,  4.,  2.]],

       [[ 7., 11.,  8., 12., 12.],
        [ 2.,  3.,  6.,  4.,  0.],
        [ 5.,  0.,  6.,  2.,  4.],
        [ 4.,  2.,  3.,  2.,  1.],
        [ 6.,  4.,  4.,  4.,  5.]],

       [[ 9., 11.,  9.,  7., 13.],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan]]])

In [32]:
from syft import PhiTensor

In [33]:
pt = PhiTensor(np.ones((5,5)), data_subjects=DataSubjectList.from_objs(np.ones((5,5))), 
    min_vals=lazyrepeatarray(0, (5,5)), 
    max_vals=lazyrepeatarray(1, (5,5)))
pt2 = PhiTensor(np.ones((5,5)) * 9, data_subjects=DataSubjectList.from_objs(np.zeros((5,5))), 
    min_vals=lazyrepeatarray(0, (5,5)), 
    max_vals=lazyrepeatarray(1, (5,5)))

In [34]:
pt[0] = pt2[0]

(5, 5) (5,)
first dim 5
shapes:  (5, 5) (5,)
len:  5 5
(5,)


In [35]:
pt

PhiTensor(child=[[9. 9. 9. 9. 9.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]], min_vals=<lazyrepeatarray data: 0 -> shape: (5, 5)>, max_vals=<lazyrepeatarray data: 9.0 -> shape: (5, 5)>)

In [36]:
pt.gamma

GammaTensor(child=array([[9., 9., 9., 9., 9.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]]), data_subjects=<syft.core.adp.data_subject_list.DataSubjectList object at 0x7fc8e5f6f580>, min_vals=<lazyrepeatarray data: 0 -> shape: (5, 5)>, max_vals=<lazyrepeatarray data: 9.0 -> shape: (5, 5)>, is_linear=True, func=<function no_op at 0x7fc8ec00b790>, id='1761323026', state={})

In [37]:
pt.data_subjects.one_hot_lookup

array([1., 0.])